### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [8]:
import sys
import os


path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [9]:
from diffusion.data_loaders.motion_dataset_v2 import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_cartwheel.txt", shuffle=True)
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.85536, 0.9966429999999997, -0.0070009999999999795, 0.08157, 0.0005729999999999971, 0.042303731260289315, -0.056088768155961526, -0.01172717680484046, -0.014103614145860938, 0.2358842735659614, 0.37124889801787253, -0.6111023347690597, -0.09268300376873025, -0.09541896434572254, 0.585361, 0.1699928747321186, 0.08652758875118252, 0.354108626550405, 0.160215, -0.2285399691330798, -0.39445967594673703, -0.1178224382194308, -0.369571, 0.20448116583595066, -0.12115992907931128, 0.07892319943485762, 0.3736623102073797, -0.010008232584494297, 0.30603690929303384, -0.364281, -0.13425257761871864, -0.004787718949892447, 0.0010873114649849894] 35
[-0.23938   2.078199 -0.008457]
[-0.23938   2.078199]


(160,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8554,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0037, -0.0062,  0.8563,  ..., -0.2435, -1.1484, -0.8920],
         [ 0.0064, -0.0122,  0.8575,  ..., -0.0278, -1.2998, -0.9550],
         ...,
         [-0.2466,  2.0540,  0.8465,  ..., -0.8048,  0.5575,  1.2816],
         [-0.2435,  2.0658,  0.8467,  ..., -0.7320,  0.5210,  1.2596],
         [-0.2394,  2.0782,  0.8469,  ..., -0.5900,  0.5328,  1.0961]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.5536e-01,  9.9664e-01, -7.0010e-03,
          8.1570e-02,  5.7300e-04,  4.2304e-02, -5.6089e-02, -1.1727e-02,
         -1.4104e-02,  2.3588e-01,  3.7125e-01, -6.1110e-01, -9.2683e-02,
         -9.5419e-02,  5.8536e-01,  1.6999e-01,  8.6528e-02,  3.5411e-01,
          1.6022e-01, -2.2854e-01, -3.9446e-01, -1.1782e-01, -3.6957e-01,
          2.0448e-01, -1.2116e-01,  7.8923e-02,  3.7366e-01, -1.0008e-02,
          3.0604e-01, -3.6428e-01, -1.3425e-01, -4.7877e-03,  1.0873e-03,
 

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [10]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-cartwheel-shuffled-128"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [11]:
from diffusion.diffuser.models.temporal_v2 import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal_v2.TemporalUnet'>
    cond_dim: 69
    horizon: 160
    transition_dim: 69

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/model_config.pkl

[ models/temporal ] Channel dimensions: [(69, 128), (128, 256), (256, 512), (512, 1024)]
[(69, 128), (128, 256), (256, 512), (512, 1024)]


In [12]:
from diffusion.diffuser.models.diffusion_v2 import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1
pos_dim = 35
vel_dim = 34

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, "diffusion_config.pkl"),
    horizon=horizon,
    # transition_dim=transition_dim,
    observation_dim=pos_dim,
    action_dim=vel_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion_v2.GaussianDiffusion'>
    action_dim: 34
    action_weight: 5
    clip_denoised: False
    horizon: 160
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    observation_dim: 35
    predict_epsilon: False

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/diffusion_config.pkl



### Setup Training

In [13]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/trainer_config.pkl



### Test if model and training loop works

In [14]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 160, 69])
x.shape torch.Size([1, 69, 160])
x.shape torch.Size([1, 128, 160])
1
xfinal.shape torch.Size([1, 128, 80])
x.shape torch.Size([1, 256, 80])
2
xfinal.shape torch.Size([1, 256, 40])
x.shape torch.Size([1, 512, 40])
3
xfinal.shape torch.Size([1, 512, 20])
x.shape torch.Size([1, 1024, 20])
4
xfinal.shape torch.Size([1, 1024, 20])
xt1.shape torch.Size([1, 1024, 20])
xt2.shape torch.Size([1, 1024, 20])
xt3.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 512, 40])
pop.shape torch.Size([1, 256, 80])


(tensor([[[ 0.0856,  0.1876,  0.1073,  ...,  0.0175,  0.0171, -0.2867],
          [ 0.1959,  0.2406,  0.0260,  ...,  0.1504,  0.0518, -0.1517],
          [-0.0330,  0.0441,  0.1546,  ..., -0.0961, -0.0580, -0.3547],
          ...,
          [-0.1711,  0.3942, -0.1396,  ...,  0.0496,  0.0114, -0.1408],
          [-0.0535,  0.0162,  0.2045,  ...,  0.0657,  0.1267,  0.0064],
          [-0.0443, -0.0081, -0.0492,  ..., -0.0372, -0.0921, -0.2271]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 160, 69]))

In [15]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 63.13 M
         downs.3.0.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 5.24 M | Conv1d(2048, 512, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 5.24 M | Conv1d(1024, 1024, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [17]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 2

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 2 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128


3000:   0.0040 | a0_loss:   0.0006 | t:   0.1876
3100:   0.0215 | a0_loss:   0.0009 | t:  12.5761
3200:   0.0170 | a0_loss:   0.0005 | t:  12.6667
3300:   0.0089 | a0_loss:   0.0009 | t:  13.0085
3400:   0.0262 | a0_loss:   0.0017 | t:  13.4865
3500:   0.0243 | a0_loss:   0.0019 | t:  12.4572
3600:   0.0016 | a0_loss:   0.0007 | t:  12.7986
3700:   0.0436 | a0_loss:   0.0024 | t:  12.6441
3800:   0.0212 | a0_loss:   0.0011 | t:  12.5137
3900:   0.0106 | a0_loss:   0.0007 | t:  12.6441
Epoch 1 / 2 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/state_0.pt
4000:   0.0033 | a0_loss:   0.0004 | t:   0.7441
4100:   0.0303 | a0_loss:   0.0038 | t:  13.1800
4200:   0.0021 | a0_loss:   0.0005 | t:  12.8341
4300:   0.0144 | a0_loss:   0.0012 | t:  13.3843
4400:   0.0017 | a0_loss:   0.0004 | t:  12.9846
4500:   0.0112 | a0_loss:   0.0004 | t:  12.5400
4600:

### (Alternatively) load a checkpoint

In [7]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(5)
model = trainer.ema_model

### Sample from model - With more frames than training

Start from 2x the training length and see if it can generate the same motion twice or longer

In [20]:
from diffuser.utils import batchify

test = dataset[0]
batch = batchify(test)

# Triple the number of frames for the noise
modified_shape_list = list(batch.trajectories.shape)
modified_shape_list[1] = modified_shape_list[1] * 3
modified_shape_tuple = torch.Size(modified_shape_list)

sample = diffusion.p_sample_loop(
    modified_shape_tuple, batch.conditions
)


                                                                                                    
1 / 1000 [                                                            ]   0% | 7.1 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 13.0 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 17.8 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                               

### Save results to logs

In [21]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0)[:, :35].cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="long-form-motion1.npy")

Motion saved as long-form-motion1.npy


Alternatively, we can place checkpoints in the long cartwheel motion

In [22]:
from diffuser.utils import batchify

test = dataset[0]
batch = batchify(test)
multiplier = 3

consequent_cartwheels = dataset[0].trajectories.repeat(3, 1)

# End and start cartwheel pos diff
diff = consequent_cartwheels[159, :] - consequent_cartwheels[0, :]
print(diff[:3])

# Account for position of consequent frames, these makes the 2nd and 3rd cartwheels start at a position where the previous cartwheel ended
consequent_cartwheels[160:, :3] += diff[:3]
consequent_cartwheels[320:, :3] += diff[:3]


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = consequent_cartwheels[:, :35]
    pos_data = pos_data.cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")
save_motions(consequent_cartwheels, f"{savepath}/sampled_motions", filename="consequent_cartwheels.npy")


consequent_cartwheels = consequent_cartwheels.unsqueeze(0)
consequent_cartwheels.shape

tensor([-0.2394,  2.0782, -0.0085])
Motion saved as consequent_cartwheels.npy


torch.Size([1, 480, 69])

Determine the peaks of the cartwheel

In [29]:
window = 30

first_checkpoint_idx = 200
second_checkpoint_idx = 300

def apply_conditioning_checkpoints(x):
    x[:, first_checkpoint_idx - window : first_checkpoint_idx + window, 3:] = consequent_cartwheels[
        :, first_checkpoint_idx - window : first_checkpoint_idx + window, 3:
    ]
    x[:, second_checkpoint_idx - window : second_checkpoint_idx + window, 3:] = consequent_cartwheels[
        :, second_checkpoint_idx - window : second_checkpoint_idx + window, 3:
    ]
    return x

In [30]:
sample = diffusion.p_sample_loop(
    consequent_cartwheels.shape, batch.conditions, max_timesteps=1000, conditioning_fn=apply_conditioning_checkpoints
)


                                                                                                    
1 / 1000 [                                                            ]   0% | 74.4 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 74.1 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 73.6 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                              

In [31]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0)[:, :35]
    # pos_data[:, 2] -= 1
    pos_data = pos_data.cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="long-form-with-checkpoints-motion.npy")

Motion saved as long-form-with-checkpoints-motion.npy


In [33]:
d = dataset[0].trajectories
d[0][0]
print(d[0][:5])
print(d[1][:5])
print(d[2][:5])
print(d[3][:5])
print(d[5][:5])
print(d[50][:5])
print(d[100][:5])
print(d[159][:5])

tensor([ 0.0000,  0.0000,  0.8554,  0.9966, -0.0070])
tensor([ 0.0037, -0.0062,  0.8563,  0.9967, -0.0061])
tensor([ 0.0064, -0.0122,  0.8575,  0.9967, -0.0046])
tensor([ 0.0088, -0.0175,  0.8586,  0.9967, -0.0026])
tensor([ 0.0119, -0.0257,  0.8610,  0.9968,  0.0025])
tensor([ 0.1900,  0.2118,  0.7810,  0.6991, -0.4624])
tensor([-0.2376,  1.4167,  0.8857, -0.3775, -0.8155])
tensor([-0.2394,  2.0782,  0.8469,  0.9929,  0.0204])
